In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [2]:
list_real = []
list_fake = []
import os
for a,b,c in os.walk('datapath'):
    for _c in c:
        if '.jpg' in _c or '.png' in _c:
            if '/real' in a:
                list_real.append(os.path.join(a,_c))
            else:
                list_fake.append(os.path.join(a,_c))

In [6]:
import timm
import torch
model = timm.create_model("hf_hub:timm/mobilenetv4_conv_small.e2400_r224_in1k", num_classes=1, pretrained=False).cuda()
weight = 'model_best_val1_hter.pth.tar'
model.load_state_dict(torch.load(weight)['state_dict'])

<All keys matched successfully>

In [ ]:
all_list = list_real+list_fake
import random
random.shuffle(all_list)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy
target_layers = [model.blocks, model.conv_stem, model.norm_head]
for i, imgpath in enumerate(all_list):
    # ten_img_rgb = torch.tensor(cv2.imread(img))
    img = cv2.imread(imgpath)
    img = cv2.resize(img, (92,92))
    img = (img/255.).astype(np.float32)
    img = img.transpose((2,0,1))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_rgb = copy.deepcopy(img)
    img_rgb[0] = (img_rgb[0] - mean[0]) / std[0] 
    img_rgb[1] = (img_rgb[1] - mean[1]) / std[1] 
    img_rgb[2] = (img_rgb[2] - mean[2]) / std[2] 
    t = 0 if '/real/' in imgpath else 1 # if classes is only two
    t = 0 # if classes is only one
    targets = [ClassifierOutputTarget(t)]
    with GradCAM(model=model, target_layers=target_layers) as cam:
        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cam = cam(input_tensor=torch.tensor(img_rgb).unsqueeze(0), targets=targets)
        # In this example grayscale_cam has only one image in the batch:
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(np.array(torch.tensor(img).permute((1,2,0)).cpu().detach()), grayscale_cam, use_rgb=True)
        # You can also get the model outputs without having to redo inference
        model_outputs = cam.outputs
        print(imgpath)
        plt.figure(figsize=(3, 2))
        plt.imshow(visualization)
        plt.show()
        if i == 20:
            break